In [1]:
import pandas as pd 
import numpy as np
data = pd.read_feather('tavbase/gs.feather')

In [7]:
def zscore(df, grupo, var, score):
    media = df[var].mean()
    dpadr = df[var].std()
    gr = df.groupby(grupo)[var]
    dc = gr.describe()
    dc = dc.reset_index()
    dc[score] = dc['mean'].apply(lambda x : (x - media) / dpadr)
    dcz = dc[[grupo, score]]
    df = df.merge(dcz, on=grupo, how='left')
    return df

In [15]:
## Classificação zscore
dimensao = 'Customer ID'
medidas = ['Sales', 'Profit', 'Quantity']
grupo = data.groupby(dimensao)[medidas].mean().reset_index()
for medida in medidas:
    grupo = zscore(grupo, dimensao, medida, 'z'+medida)
zmedidas = ['z'+med for med in medidas]
zgrupo = grupo[zmedidas]
grupo['score'] = zgrupo.sum(axis=1)
media_score = grupo['score'].mean()
dpadr_score = grupo['score'].std()
grupo['classe'] = grupo['score'].apply(
    lambda x : int((x - media_score) / dpadr_score) + 3
)
grupo['rank'] = grupo['score'].rank(ascending=False)
grupo['lucro'] = grupo['Profit'].apply(lambda x : 0 if x < 0 else 1)
grupo

,Customer ID,Sales,Profit,Quantity,zSales,zProfit,zQuantity,score,classe,rank,lucro
0,AA-10315,327.319357,10.659298,3.452381,1.070609,-0.286579,0.449566,1.233595,3,506.0,1
1,AA-10375,140.099881,16.130414,3.309524,-0.712586,-0.161740,0.296825,-0.577500,3,979.0,1
2,AA-10480,465.673415,39.907242,3.947368,2.388379,0.380799,0.978798,3.747975,4,56.0,1
3,AA-10645,210.190284,41.800534,3.657534,-0.045001,0.424000,0.668912,1.047910,3,575.0,1
4,AA-315,280.407000,66.945750,2.500000,0.623786,0.997762,-0.568703,1.052845,3,573.0,1
...,...,...,...,...,...,...,...,...,...,...,...
1585,YS-21880,346.363074,57.251746,3.648148,1.251993,0.776564,0.658877,2.687434,4,159.0,1
1586,ZC-11910,7.173000,-15.567000,1.000000,-1.978664,-0.885009,-2.172477,-5.036151,1,1580.0,0
1587,ZC-21910,338.962134,5.386944,3.964286,1.181502,-0.406883,0.996885,1.771504,3,337.0,1
1588,ZD-11925,163.957000,26.578667,3.055556,-0.485356,0.076668,0.025287,-0.383401,3,947.0,1


In [ ]:
grupo.to_feather('tavbase/classificacaoz_consumidor.feather')